In [2]:
from __future__ import print_function

import os
from skimage.transform import resize
from skimage.io import imsave, imread
import numpy as np
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.utils import to_categorical
from keras import backend as K
from sklearn.model_selection import train_test_split

K.set_image_data_format('channels_last') 

data_path = 'raw/'

image_rows = 256
image_cols = 256
n_classes = 8
smooth = 1.

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)


Using TensorFlow backend.


In [2]:
train_data_path = os.path.join(data_path, 'train')
annotation_data_path = os.path.join(data_path, 'annotations')

images = os.listdir(train_data_path)
total = len(images)

imgs = np.ndarray((total, image_rows, image_cols, 3), dtype=np.float32)
imgs_mask = np.ndarray((total, image_rows, image_cols), dtype=np.byte)

i = 0
print('-'*30)
print('Creating training images...')
print('-'*30)
for image_name in images:
    img = imread(os.path.join(train_data_path, image_name))
    img_mask = imread(os.path.join(annotation_data_path, image_name), as_gray=True)

    img = np.array([img])
    img_mask = np.array([img_mask])

    imgs[i] = img
    imgs_mask[i] = img_mask

    if i % 100 == 0:
        print('Done: {0}/{1} images'.format(i, total))
    i += 1
print('Loading done.')

------------------------------
Creating training images...
------------------------------
Done: 0/4291 images
Done: 100/4291 images
Done: 200/4291 images
Done: 300/4291 images
Done: 400/4291 images
Done: 500/4291 images
Done: 600/4291 images
Done: 700/4291 images
Done: 800/4291 images
Done: 900/4291 images
Done: 1000/4291 images
Done: 1100/4291 images
Done: 1200/4291 images
Done: 1300/4291 images
Done: 1400/4291 images
Done: 1500/4291 images
Done: 1600/4291 images
Done: 1700/4291 images
Done: 1800/4291 images
Done: 1900/4291 images
Done: 2000/4291 images
Done: 2100/4291 images
Done: 2200/4291 images
Done: 2300/4291 images
Done: 2400/4291 images
Done: 2500/4291 images
Done: 2600/4291 images
Done: 2700/4291 images
Done: 2800/4291 images
Done: 2900/4291 images
Done: 3000/4291 images
Done: 3100/4291 images
Done: 3200/4291 images
Done: 3300/4291 images
Done: 3400/4291 images
Done: 3500/4291 images
Done: 3600/4291 images
Done: 3700/4291 images
Done: 3800/4291 images
Done: 3900/4291 images
Do

In [3]:
imgs_train, imgs_test, imgs_mask_train, imgs_mask_test = train_test_split(imgs, imgs_mask, test_size=0.10, random_state=42)

In [4]:
np.save('imgs_train.npy', imgs_train)
np.save('imgs_mask_train.npy', imgs_mask_train)
np.save('imgs_test.npy', imgs_test)
np.save('imgs_mask_test.npy', imgs_mask_test)

In [6]:
del imgs_train
del imgs_mask_train
del imgs_test
del imgs_mask_test

In [2]:
imgs_train = np.load('imgs_train.npy')
imgs_mask_train = np.load('imgs_mask_train.npy')

mean = np.mean(imgs_train)  # mean for data centering
std = np.std(imgs_train)  # std for data normalization

imgs_train -= mean
imgs_train /= std

imgs_mask_train = to_categorical(imgs_mask_train, num_classes=n_classes)

In [3]:
def UNet(input_shape=(256, 256, 3), classes=1):
    inputs = Input(shape=input_shape)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    pool1 = Dropout(0.25)(pool1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    pool2 = Dropout(0.5)(pool2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    pool3 = Dropout(0.5)(pool3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    pool4 = Dropout(0.5)(pool4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    up6 = Dropout(0.5)(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    up7 = Dropout(0.5)(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    up8 = Dropout(0.5)(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    up9 = Dropout(0.5)(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(classes, (1, 1), activation='softmax')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])
    
    return model

In [4]:
model = UNet(classes=n_classes)
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=[dice_coef])

early_stopping = EarlyStopping(patience=10, verbose=1)
model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss', save_best_only=True)
reduce_lr = ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=1)
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)

In [5]:
print('-'*30)
print('Fitting model...')
print('-'*30)
model.fit(imgs_train, imgs_mask_train, batch_size=12, epochs=100, verbose=1, shuffle=True,
          validation_split=0.1,
          callbacks=[model_checkpoint, early_stopping, reduce_lr, tensorboard])

------------------------------
Fitting model...
------------------------------
Train on 3474 samples, validate on 387 samples
Epoch 1/100
3474/3474 [==============================] - 94s 27ms/step - loss: 0.5352 - dice_coef: 0.7559 - val_loss: 0.4189 - val_dice_coef: 0.7804
Epoch 2/100
3474/3474 [==============================] - 88s 25ms/step - loss: 0.3958 - dice_coef: 0.8038 - val_loss: 0.3726 - val_dice_coef: 0.7873
Epoch 3/100
3474/3474 [==============================] - 88s 25ms/step - loss: 0.3570 - dice_coef: 0.8193 - val_loss: 0.3462 - val_dice_coef: 0.8230
Epoch 4/100
3474/3474 [==============================] - 99s 28ms/step - loss: 0.3348 - dice_coef: 0.8289 - val_loss: 0.3795 - val_dice_coef: 0.8325
Epoch 5/100
3474/3474 [==============================] - 89s 26ms/step - loss: 0.3151 - dice_coef: 0.8373 - val_loss: 0.3224 - val_dice_coef: 0.8304
Epoch 6/100
3474/3474 [==============================] - 90s 26ms/step - loss: 0.3085 - dice_coef: 0.8406 - val_loss: 0.3037 - va


Epoch 00053: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 54/100
3474/3474 [==============================] - 88s 25ms/step - loss: 0.2084 - dice_coef: 0.8853 - val_loss: 0.2278 - val_dice_coef: 0.8864
Epoch 55/100
3474/3474 [==============================] - 88s 25ms/step - loss: 0.2081 - dice_coef: 0.8853 - val_loss: 0.2275 - val_dice_coef: 0.8862
Epoch 56/100
3474/3474 [==============================] - 88s 25ms/step - loss: 0.2083 - dice_coef: 0.8853 - val_loss: 0.2279 - val_dice_coef: 0.8865
Epoch 57/100
3474/3474 [==============================] - 88s 25ms/step - loss: 0.2081 - dice_coef: 0.8850 - val_loss: 0.2276 - val_dice_coef: 0.8866
Epoch 58/100
3474/3474 [==============================] - 88s 25ms/step - loss: 0.2081 - dice_coef: 0.8855 - val_loss: 0.2279 - val_dice_coef: 0.8866
Epoch 00058: early stopping


In [6]:
del imgs_train
del imgs_mask_train

In [5]:
imgs_test = np.load('imgs_test.npy')
imgs_mask_test = np.load('imgs_mask_test.npy')

mean = np.mean(imgs_test)
std = np.std(imgs_test)

imgs_test -= mean
imgs_test /= std

imgs_mask_test = to_categorical(imgs_mask_test, num_classes=8)

In [6]:
model.load_weights('weights.h5')

In [7]:
imgs_mask_predict = model.predict(imgs_test, verbose=1)

430/430 [==============================] - 6s 15ms/step


In [11]:
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches

from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})

def discrete_cmap(N, base_cmap=None):
    """Create an N-bin discrete colormap from the specified input map"""

    # Note that if base_cmap is a string or None, you can simply do
    #    return plt.cm.get_cmap(base_cmap, N)
    # The following works for string, None, or a colormap instance:

    base = plt.cm.get_cmap(base_cmap)
    color_list = base(np.linspace(0, 1, N))
    cmap_name = base.name + str(N)
    return base.from_list(cmap_name, color_list, N)

pred_dir = 'preds'
if not os.path.exists(pred_dir):
    os.mkdir(pred_dir)
    
imgs_test = np.load('imgs_test.npy')

In [ ]:
plt.rc('text', usetex=True)

for img_id in range(len(imgs_mask_predict)):
    f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(ncols=2,nrows=2,figsize=(8,8))

    labels = [r'FLORESTA', r'DESMATAMENTO', r'HIDROGRAFIA', r'RESIDUO', r'NUVEM', r'NAO\_FLORESTA2', r'NAO\_FLORESTA']

    c = plt.get_cmap('jet', n_classes)

    img_mask_test = np.argmax(imgs_mask_test[img_id], axis = 2)
    img_mask_predict = np.argmax(imgs_mask_predict[img_id], axis = 2)

    im1 = ax1.imshow(imgs_test[img_id].astype(np.uint32))
    ax1.set_title(r'\centering\sffamily\bfseries (a) Imagem Original', x=.5, y=-.15)

    im2 = ax2.imshow(img_mask_test, cmap=c).set_clim(0, n_classes - 1)
    ax2.set_title(r'\centering\sffamily\bfseries (b) Label Original', x=.5, y=-.15)
    
    im3 = ax3.imshow(imgs_test[img_id].astype(np.uint32), alpha=0.5)
    ax3.imshow(np.argmax(imgs_mask_predict[img_id], axis = 2), alpha=0.7, cmap='gray') # OVERLAY
    ax3.set_title(r'\centering\sffamily\bfseries (c) Imagem Original + Label Rede Neural', x=.5, y=-.15)

    im4 = ax4.imshow(img_mask_predict, cmap=c).set_clim(0, n_classes - 1)
    
    ax4.set_title(r'\centering\sffamily\bfseries (d) Label Rede Neural', x=.5, y=-.15)
    
    colors = [c(value + 1) for value in np.arange(0, n_classes)]
    patches = [ mpatches.Patch(color=colors[i], label="{l}".format(l=labels[i]) ) for i in range(len(labels)) ]
    
    plt.draw()



    lgd = f.legend(borderaxespad=0, handles=patches, loc='center')

    bb = lgd.get_bbox_to_anchor().inverse_transformed(ax2.transAxes)
    xOffset = 1.5
    bb.x0 += xOffset
    bb.x1 += xOffset
    lgd.set_bbox_to_anchor(bb, transform = ax2.transAxes)

    plt.tight_layout()

    f.savefig('graphs/graph_{}.png'.format(img_id), format='png', bbox_extra_artists=(lgd,), bbox_inches='tight', dpi=300)
    plt.close(f)

c:\program files\python36\lib\site-packages\matplotlib\font_manager.py:1328: UserWarning: findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


In [112]:

# i=0
# for image_pred, image_mask_test, image_test in zip(imgs_mask_predict, imgs_mask_test, imgs_test): 
#     imsave(os.path.join(pred_dir, str(i) +'.png'), image_test.astype(np.uint8))
#     imsave(os.path.join(pred_dir, str(i) +'_label.png'), np.argmax(image_mask_test, axis = 2).astype(np.uint8) * (255 // n_classes))
#     imsave(os.path.join(pred_dir, str(i) +'_pred.png'), np.argmax(image_pred, axis = 2) * (255// n_classes))
#     i += 1

c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\0_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\1_label.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\1_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\2_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\3_label.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\3_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:

c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\41_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\42_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\43_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\44_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\45_label.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\45_pred.png is a low contrast image
  warn('%s is a low contrast image' % fnam

c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\79_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\80_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\81_label.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\81_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\82_label.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\82_pred.png is a low contrast image
  warn('%s is a low contrast image' % fna

c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\117_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\118_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\119_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\120_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\121_label.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\121_pred.png is a low contrast image
  warn('%s is a low contrast image' 

c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\162_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\163_label.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\163_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\164_label.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\164_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\165_pred.png is a low contrast image
  warn('%s is a low contrast image'

...

c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\244_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\245_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\246_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\247_label.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\247_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\248_label.png is a low contrast image
  warn('%s is a low contrast image'

c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\280_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\281_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\282_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\283_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\284_label.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\284_pred.png is a low contrast image
  warn('%s is a low contrast image' 

c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\316_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\317_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\318_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\319_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\320_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\321_pred.png is a low contrast image
  warn('%s is a low contrast image' %

c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\359_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\360_label.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\360_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\361_label.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\361_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\362_pred.png is a low contrast image
  warn('%s is a low contrast image'

c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\401_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\402_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\403_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\404_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\405_label.png is a low contrast image
  warn('%s is a low contrast image' % fname)
c:\program files\python36\lib\site-packages\skimage\io\_io.py:140: UserWarning: preds\405_pred.png is a low contrast image
  warn('%s is a low contrast image' 